In [1]:
import numpy as np

from sympy import (Matrix,
                   sin,
                   cos,
                   symbols,
                   diff,
                   simplify,
                   Eq,
                   Function,
                   linear_eq_to_matrix)
from sympy.physics.mechanics import dynamicsymbols
from sympy import default_sort_key 

# Dynamic System Derivation

In [2]:
C = lambda th: Matrix([[cos(th), -sin(th)], [sin(th), cos(th)]])
D = lambda th: Matrix([[sin(th), cos(th)], [-cos(th), sin(th)]])

In [ ]:
t = symbols('t')

# Define states and their time derivatives
th_s, th_1, th_2, th_t = dynamicsymbols('theta_s theta_1 theta_2 theta_T')
dth_s, dth_1, dth_2, dth_t = dynamicsymbols('theta_s theta_1 theta_2, theta_T', 1)
ddth_s, ddth_1, ddth_2, ddth_t = dynamicsymbols('theta_s theta_1 theta_2, theta_T', 2)

# Generalized coordinates
q = Matrix([th_s, th_1, th_2, th_t])
qdot = Matrix([dth_s, dth_1, dth_2, dth_t])
qddot = Matrix([ddth_s, ddth_1, ddth_2, ddth_t])

# State vector
y = q.row_insert(4, qdot)
ydot = qdot.row_insert(4, qddot)

# Controls
tau_s = symbols('tau_s')
tau_1 = symbols('tau_1')
tau_2 = symbols('tau_2')
tau_t = symbols('tau_t')
tau = Matrix([tau_s, tau_1, tau_2, tau_t])

# System parameters
rho = symbols('rho_s rho_1 rho_2 rho_t')    # dimensional array
m = symbols('m_s m_1 m_2 m_t')              # mass array
I = symbols('I_s I_1 I_2 I_t')              # inertia array
d = symbols('d_s d_1 d_2 d_t')              # disturbance vector

# Satellite central position
r_s0, r_s1 = symbols('r_s0 r_s1')
r_s = Matrix([r_s0, r_s1])

# Target central position
r_t0, r_t1 = symbols('r_s0 r_s1')
r_t = Matrix([r_t0, r_t1])

# Mid-link positions
r_1 = r_s + C(y[0])@(Matrix([rho[0], 0.]) + C(y[1])@Matrix([rho[1], 0.]))
r_2 = r_1 + C(y[0])@C(y[1])@(Matrix([rho[1], 0.]) + C(y[2])@Matrix([rho[2], 0.]))

# Link velocities
v_1 = Matrix(diff(r_1, t))
v_2 = Matrix(diff(r_2, t))

# Define Lagrangian
T = 0.5*m[1]*v_1.dot(v_1) + 0.5*m[2]*v_2.dot(v_2) + \
    0.5*I[0]*y[4]**2 + 0.5*I[1]*y[5]**2 + \
    0.5*I[2]*y[6]**2
V = 0.

L = simplify(T - V)

In [ ]:
dL_dth_s = diff(L, y[0])
dL_dth_1 = diff(L, y[1])
dL_dth_2 = diff(L, y[2])

dL_ddth_s = diff(L, y[4])
dL_ddth_1 = diff(L, y[5])
dL_ddth_2 = diff(L, y[6])

dL_ddth_s_dt = diff(dL_ddth_s, t)
dL_ddth_1_dt = diff(dL_ddth_1, t)
dL_ddth_2_dt = diff(dL_ddth_2, t)

th_s_eqn = Eq(simplify(dL_ddth_s_dt - dL_dth_s), tau[0] - d[0])
th_1_eqn = Eq(simplify(dL_ddth_1_dt - dL_dth_1), tau[1] - d[1])
th_2_eqn = Eq(simplify(dL_ddth_2_dt - dL_dth_2), tau[2] - d[2])
th_t_eqn = Eq(I[3]*qddot[3], tau[3] - d[3])

In [ ]:
eqs = [th_s_eqn.lhs - th_s_eqn.rhs,
       th_1_eqn.lhs - th_1_eqn.rhs,
       th_2_eqn.lhs - th_2_eqn.rhs]

A, B = linear_eq_to_matrix(eqs, qddot[0], qddot[1], qddot[2], qddot[3])

# Other Important Parameters
- [x] end effector position
- [x] end effector velocity
- [x] Jacobian of end effector
- [x] Jacobian of end eddfector's time derivative
- [x] critical point positions
- [x] critical point velocities
- [x] capture point position
- [x] capture point velocity
- [x] capture point acceleration
- [x] obstacle point positions
- [x] obstacle point velocities
- [x] $\gamma_1$
- [x] $\gamma_2$
- [x] $\gamma$
- [x] $\mathbf{\lambda}$
- [x] $\mathbf{\dot{\lambda}}$

In [ ]:
# End effector position
r_ee = r_1 + C(y[0])@C(y[1])@(Matrix([rho[1], 0.]) + C(y[2])@Matrix([2.*rho[2], 0.]))

# End effector velocity
v_ee = diff(r_ee, t)

# End effector jacobian
J = r_ee.jacobian(q[:-1])

# End effector
J_dot = simplify(diff(J, t))

In [ ]:
# Critical point positions
cp_1 = r_s + C(y[0])@(Matrix([rho[0], 0.]) + C(y[1])@Matrix([2.*rho[1], 0.]))
cp_2 = r_2
cp_3 = r_ee

# Critical point velocities
v_cp_1 = diff(cp_1, t)
v_cp_2 = v_2
v_cp_3 = v_ee

In [ ]:
# Capture point position
r_c = r_t + C(y[3])@Matrix([0., rho[3]])

# Capture point velocity
v_c = Matrix(diff(r_c, t))

# Capture point acceleration
a_c = diff(v_c, t)

In [ ]:
# Obstacle point positions
op_1 = r_t + C(y[3])@Matrix([-rho[3], 0.])
op_2 = r_t + C(y[3])@Matrix([-rho[3], rho[3]])
op_3 = r_t + C(y[3])@Matrix([rho[3], rho[3]])
op_4 = r_t + C(y[3])@Matrix([rho[3], 0.])
op_5 = r_t + C(y[3])@Matrix([rho[3], -rho[3]])
op_6 = r_t + C(y[3])@Matrix([0., -rho[3]])
op_7 = r_t + C(y[3])@Matrix([-rho[3], -rho[3]])

# Obstacle point velocities
v_op_1 = diff(op_1, t)
v_op_2 = diff(op_2, t)
v_op_3 = diff(op_3, t)
v_op_4 = diff(op_4, t)
v_op_5 = diff(op_5, t)
v_op_6 = diff(op_6, t)
v_op_7 = diff(op_7, t)

In [ ]:
# Gamma 1
p = symbols('p_1 p_2 p_3')
q_bar = symbols('q_bar_1 q_bar_2 q_bar_3')
q_max = symbols('q_max_1 q_max_2 q_max_3')
q_min = symbols('q_min_1 q_min_2 q_min_3')

gam_1 = (1./3.)*(p[0]*((q[0] - q_bar[0])/(q_max[0] - q_min[0]))**2 + \
                 p[1]*((q[1] - q_bar[1])/(q_max[1] - q_min[1]))**2 + \
                 p[2]*((q[2] - q_bar[2])/(q_max[2] - q_min[2]))**2)

In [ ]:
# Gamma 2
cp_x = [cp_1[0], cp_2[0], cp_3[0]]
cp_y = [cp_1[1], cp_2[1], cp_3[1]]

op_x = [[Function(f"xT_{j+1}_{k+1}")(t) for k in range(2)] for j in range(3)]
op_y = [[Function(f"yT_{j+1}_{k+1}")(t) for k in range(2)] for j in range(3)]

w11, w12, w21, w22, w31, w32 = symbols('w11 w12 w21 w22 w31 w32', real=True)
weights = {(0, 0): w11, (0, 1): w12,
           (1, 0): w21, (1, 1): w22,
           (2, 0): w31, (2, 1): w32}

gam_2 = 0
for j in range(3):
    for k in range(2):
        d_jk_sq = (cp_x[j] - op_x[j][k])**2 + (cp_y[j] - op_y[j][k])**2
        gam_2 += weights[(j, k)] / d_jk_sq

In [ ]:
gam = Matrix([gam_1 + gam_2])

In [ ]:
lam = gam.jacobian(q[:-1])

In [ ]:
lam_dot = diff(lam, t)

In [ ]:
subs_dict = {}
velocity_symbols = {}  # To store the order of our velocity symbols if needed.
for j in range(3):
    for k in range(2):
        vx_sym = symbols(f"vxT_{j+1}_{k+1}", real=True)
        vy_sym = symbols(f"vyT_{j+1}_{k+1}", real=True)
        velocity_symbols[(j, k, 'x')] = vx_sym
        velocity_symbols[(j, k, 'y')] = vy_sym
        subs_dict[diff(op_x[j][k], t)] = vx_sym
        subs_dict[diff(op_y[j][k], t)] = vy_sym

lam_dot = lam_dot.subs(subs_dict)

In [ ]:
vars = sorted((lam_dot.free_symbols), key=default_sort_key)

Still need forward dynamics (calculate torques given states and desired accelerations)

# Create Functions

In [48]:
from sympy import lambdify, matrix2numpy

# Saving dynamics functions here (forward and inverse)
# dynamics = lambdify((y, tau, r_s, r_t, rho, m, I, d), matrix2numpy(A.LUsolve(B).T), "numpy")

lam_dot_func = lambdify(vars, matrix2numpy(lam_dot), "numpy")

# pos_func = lambdify((y, r_s, r_t, rho), (matrix2numpy(r_1.T), matrix2numpy(r_2.T),  matrix2numpy(r_c.T)), "numpy")
# vel_func = lambdify((y, r_s, r_t, rho), (matrix2numpy(v_1.T), matrix2numpy(v_2.T),  matrix2numpy(v_c.T)), "numpy")
# ee_func = lambdify((y, r_s, rho), matrix2numpy(r_ee.T), "numpy")
# J_func = lambdify((y, rho), matrix2numpy(J), "numpy")

NotImplementedError: unhandled type: <class 'numpy.ndarray'>, [[0.666666666666667*p_1*Derivative(theta_s(t), t)/(q_max_1 - q_min_1)**2 + 0.0625*w11*(-4.0*(-2*rho_1*sin(theta_1(t))*sin(theta_s(t)) + 1.0*(2.0*rho_1*cos(theta_1(t)) + rho_s)*cos(theta_s(t)))*(0.5*r_s1 + rho_1*sin(theta_1(t))*cos(theta_s(t)) + 0.5*(2.0*rho_1*cos(theta_1(t)) + rho_s)*sin(theta_s(t)) - 0.5*yT_1_1(t)) - 4.0*(-2*rho_1*sin(theta_1(t))*cos(theta_s(t)) - 1.0*(2.0*rho_1*cos(theta_1(t)) + rho_s)*sin(theta_s(t)))*(0.5*r_s0 - rho_1*sin(theta_1(t))*sin(theta_s(t)) + 0.5*(2.0*rho_1*cos(theta_1(t)) + rho_s)*cos(theta_s(t)) - 0.5*xT_1_1(t)))*(-2*(0.5*r_s0 - rho_1*sin(theta_1(t))*sin(theta_s(t)) + 0.5*(2.0*rho_1*cos(theta_1(t)) + rho_s)*cos(theta_s(t)) - 0.5*xT_1_1(t))*(-2.0*rho_1*sin(theta_1(t))*cos(theta_s(t))*Derivative(theta_1(t), t) - 2*rho_1*sin(theta_1(t))*cos(theta_s(t))*Derivative(theta_s(t), t) - 2*rho_1*sin(theta_s(t))*cos(theta_1(t))*Derivative(theta_1(t), t) - 1.0*vxT_1_1 - 1.0*(2.0*rho_1*cos(theta_1(t)) + rho

# Testing Functions

In [8]:
y = np.zeros(8)
u = np.zeros(3)
u[1] = np.deg2rad(30.)

r_s = np.array([[0.], [0.]])
r_t = np.array([[2.], [0.]])

rho = np.array([0.5, 0.5, 0.5, 0.5])
m = np.array([250., 25., 25., 180.])
I = np.array([25., 2.5, 2.5, 18.])
b = np.zeros(3)

print(dynamics(y, u, r_s, r_t, rho, m, I, b))
print(pos_func(y, r_s, r_t, rho))
print(vel_func(y, r_s, r_t, rho))

[[-0.02223101  0.05148234 -0.04680213  0.        ]]
(array([[[3.],
        [0.]]]), array([[[4.],
        [0.]]]), array([[[2. ],
        [0.5]]]))
(array([[-0.,  0.]]), array([[0., 0.]]), array([[-0., -0.]]))


In [10]:
print(ee_func(y, r_s, rho))

[[[2.5]
  [0. ]]]


In [11]:
print(J_func(y, rho))

[[-0.  -0.  -0. ]
 [ 2.5  2.   1. ]]


# Save Functions in PKL

In [12]:
import dill

with open('dynamics.pkl', "wb") as f:
    dill.dump(dynamics, f)

with open('position.pkl', "wb") as f:
    dill.dump(pos_func, f)

with open('velocity.pkl', "wb") as f:
    dill.dump(vel_func, f)

with open('ee.pkl', 'wb') as f:
    dill.dump(ee_func, f)

with open('Je.pkl', 'wb') as f:
    dill.dump(J_func, f)